# Get the Workspace

In [1]:
from azureml.core.workspace import Workspace

workspace = Workspace.from_config()

# Load the Model

In [9]:
from azureml.core.model import Model
import pickle

model = Model(workspace, 'churn-model')
model.download(exist_ok=True)

loaded_model = pickle.load(open('model.pickle', "rb"))

In [32]:
%%writefile entry.py
import json
import pickle
import numpy as np
import pandas as pd
import os
import xgboost as xgb

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pickle')
    print(model_path)
    model = pickle.load(open(model_path, "rb"))

# Called when a request is received
def run(data):
    data = pd.read_json(data, orient = 'split')
    data_xgb = xgb.DMatrix(data)

    # Return the prediction
    prediction = predict(data_xgb)
    return prediction

def predict(data):
    prediction = model.predict(data)[0]
    return {"churn-prediction": str(int(prediction))}

Overwriting entry.py


# Create an environment 

In [18]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

xgboost_env = Environment(workspace=workspace, name="XGBoost-Env")
conda_dep = CondaDependencies()

conda_dep.add_pip_package("azureml-dataprep[pandas]")
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("xgboost==0.90")
conda_dep.add_pip_package("scikit-learn==0.22.2.post1")
conda_dep.add_pip_package("numpy==1.18.5")

# Adds dependencies to PythonSection of myenv
xgboost_env.python.conda_dependencies=conda_dep
xgboost_env.register(workspace)

## Combine Entry Script and Environment

In [29]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="entry.py",
                                   environment=xgboost_env)

# Deploy

## Get the Inference Cluster

In [22]:
from azureml.core.compute import AksCompute

cluster_name = 'aks-e2e-1'
aks_target = AksCompute(workspace, cluster_name)

## Deploy the Model

In [30]:
from azureml.core.model import Model

model = workspace.models['churn-model']

service = Model.deploy(workspace=workspace,
                       name = 'churn-model-dev',
                       models = [model],
                       inference_config = inference_config,
                       deployment_target = aks_target,
                       overwrite=True)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-15 16:13:26+00:00 Creating Container Registry if not exists.
2021-11-15 16:13:26+00:00 Registering the environment.
2021-11-15 16:13:27+00:00 Use the existing image.
2021-11-15 16:13:28+00:00 Checking the status of deployment churn-model-dev.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 33543f92-ef65-443d-bdf6-8c7bb7df48ae
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/var/azureml-app/azureml-models/churn-model/7/outputs/model.pkl', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.m

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 33543f92-ef65-443d-bdf6-8c7bb7df48ae
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/var/azureml-app/azureml-models/churn-model/7/outputs/model.pkl', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your 